In [2]:
import os
import sys
import numpy as np
import tensorflow as tf
from tensorflow.contrib.training import bucket_by_sequence_length

tf.reset_default_graph()
BASE = '/home/brandon/terabyte/Datasets/test_data/'
file = os.path.join(BASE, 'train_from.txt.ids1000')
sess = None

omg this works


A **TFRecords** file contains a sequence of strings with CRC hashes. Each record has the format

```python
uint64 length
uint32 masked_crc32_of_length
byte   data[length]
uint32 masked_crc32_of_data
```

In [15]:
tfrecords_fname = 'test.tfrecords'
print("Currently in dir: ", os.getcwd())
train_file = os.path.join(BASE, 'train_from.txt.ids1000')
with tf.gfile.GFile(train_file, mode="r") as source_file:
    with tf.python_io.TFRecordWriter(tfrecords_fname) as writer:
        source = source_file.readline()
        while source:
            example = tf.train.SequenceExample()
            example.context.feature['length'].int64_list.value.append(len([int(x) for x in source.split()]))
            for x in source.split():
                example.feature_lists.feature_list['encoder_tokens'].feature.add().int64_list.value.append(int(x))
            writer.write(example.SerializeToString())
            #writer.write(tf.compat.as_bytes(source))
            source = source_file.readline()

Currently in dir:  /home/brandon/Documents/seq2seq_projects/notebooks


In [40]:
bucket_by_sequence_length??

In [25]:
sess = None
tf.reset_default_graph()
sess = None

In [30]:
capacity = 5
LAST_RUNNER = 'batch_dequeue'
CONTEXT_FEATURES = {'length': tf.FixedLenFeature([], dtype=tf.int64)}
SEQUENCE_FEATURES = {'encoder_tokens': tf.FixedLenSequenceFeature([], dtype=tf.int64)}
tfrecords_fname = 'test.tfrecords'

filename_queue = tf.train.string_input_producer([tfrecords_fname])
reader = tf.TFRecordReader(name='tfrecord_reader')
_, next_raw = reader.read(filename_queue, name='read_records')

q = tf.RandomShuffleQueue(capacity=capacity,min_after_dequeue=0,dtypes=tf.string,shapes=[()],name='randomize_records')
enqueue_op = q.enqueue(next_raw)
example_dq = q.dequeue()
seq_len_parsed, sequence_parsed = tf.parse_single_sequence_example(serialized=example_dq,
                                                     context_features=CONTEXT_FEATURES,
                                                     sequence_features=SEQUENCE_FEATURES)
seq_len = seq_len_parsed['length'] 
sequence = sequence_parsed['encoder_tokens']

numberOfThreads = 1
qr = tf.train.QueueRunner(q, [enqueue_op] * numberOfThreads)
# Don't forget to add your "QueueRunner" to the QUEUE_RUNNERS collection
tf.train.add_queue_runner(qr)

In [31]:
sequence_length, outputs = bucket_by_sequence_length(
    input_length=tf.to_int32(seq_len),
    tensors=[sequence],
    batch_size=2, bucket_boundaries=[2, 5], capacity=5,
    dynamic_pad=True, name='bucket_by_sequence_length', 
)

In [32]:
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    fetched = sess.run({'out_len': sequence_length, 'out': outputs})
    print('ay')
    print(fetched['out_len'])
    print(fetched['out'])
    coord.request_stop()
    coord.join(threads)

ay
[9 6]
[array([[10,  7,  6, 14, 23,  6, 31, 19,  4],
       [17,  8, 22, 43, 28,  4,  0,  0,  0]])]
